# Extracción de características *Bag of Words*

Primero importamos todas las librerías necesarias

In [1]:
import pandas as pd
import numpy as np
import re
import string
import spacy
import gensim

pd.options.display.max_colwidth = None


Creamos un pequeño cuerpo de textos de ejemplo *(CORPUS)*

In [2]:
corpus = ['El cielo es azul y bonito',
          'Me encanta el cielo azul, pero no el cielo plomizo',
          'Bonito cielo hacía ese día',
          'Hoy he desayunado huevos con jamón y tostadas',
          'Juan odia las tostadas y los huevos con jamón',
          'las tostadas de jamón están muy buenas']

## Limpieza del texto
Definimos una función simple de limpieza y normalización del texto y la aplicamos a nuestro corpus.

In [3]:
nlp = spacy.load("es_core_news_md")
def normalizar_doc(doc):
    '''Función que normaliza un texto cogiendo sólo
    las palabras en minúsculas mayores de 3 caracteres'''
    # separamos en tokens
    tokens = nlp(doc)
    # filtramos stopwords
    filtered_tokens = [t.lower_ for t in tokens if
                       len(t.text)>3 and
                       not t.is_space and
                       not t.is_punct]
    # juntamos de nuevo en una cadena
    doc = ' '.join(filtered_tokens)
    return doc

In [4]:
#probamos la función
normalizar_doc(corpus[0])

'cielo azul bonito'

In [5]:
corpus[0]

'El cielo es azul y bonito'

In [6]:
#aplicamos a todo el corpus
norm_corpus = [normalizar_doc(doc) for doc in corpus]
norm_corpus

['cielo azul bonito',
 'encanta cielo azul pero cielo plomizo',
 'bonito cielo hacía',
 'desayunado huevos jamón tostadas',
 'juan odia tostadas huevos jamón',
 'tostadas jamón están buenas']

In [7]:
#alternativamente
list(map(normalizar_doc, corpus))

['cielo azul bonito',
 'encanta cielo azul pero cielo plomizo',
 'bonito cielo hacía',
 'desayunado huevos jamón tostadas',
 'juan odia tostadas huevos jamón',
 'tostadas jamón están buenas']

# Librería `scikit-learn`
Implementamos el modelo Bag-of-Word (BoW) con `scikit-learn`

Contamos la frecuencia de aparición de los términos en cada documento, usando un vocabulario común. 

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
cv.fit(norm_corpus) #también funcionaría cv.fit(map(normalizar_doc, corpus))

CountVectorizer()

In [9]:
type(cv)

sklearn.feature_extraction.text.CountVectorizer

In [10]:
dir(cv)

['__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__sklearn_clone__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_build_request_for_signature',
 '_char_ngrams',
 '_char_wb_ngrams',
 '_check_feature_names',
 '_check_n_features',
 '_check_stop_words_consistency',
 '_check_vocabulary',
 '_count_vocab',
 '_doc_link_module',
 '_doc_link_template',
 '_doc_link_url_param_generator',
 '_get_default_requests',
 '_get_doc_link',
 '_get_metadata_request',
 '_get_param_names',
 '_get_tags',
 '_limit_features',
 '_more_tags',
 '_parameter_constraints',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_sort_features',
 '_stop_words_id',
 '_validate_data',
 '_

El modelo genera un diccionario con todas las palabras del vocabulario y asigna un índice único a cada palabra:

In [11]:
cv.get_feature_names_out()

array(['azul', 'bonito', 'buenas', 'cielo', 'desayunado', 'encanta',
       'están', 'hacía', 'huevos', 'jamón', 'juan', 'odia', 'pero',
       'plomizo', 'tostadas'], dtype=object)

In [12]:
len(cv.get_feature_names_out())

15

In [13]:
cv.vocabulary_

{'cielo': 3,
 'azul': 0,
 'bonito': 1,
 'encanta': 5,
 'pero': 12,
 'plomizo': 13,
 'hacía': 7,
 'desayunado': 4,
 'huevos': 8,
 'jamón': 9,
 'tostadas': 14,
 'juan': 10,
 'odia': 11,
 'están': 6,
 'buenas': 2}

A partir del vocabulario aprendido, generamos el vector BoW de cada documento creando una matriz:

In [14]:
cv_matrix = cv.transform(norm_corpus)
cv_matrix.shape

(6, 15)

In [15]:
#matriz sparse
cv_matrix

<6x15 sparse matrix of type '<class 'numpy.int64'>'
	with 24 stored elements in Compressed Sparse Row format>

In [16]:
#sólo guarda info de las celdas no vacías
print(cv_matrix)

  (0, 0)	1
  (0, 1)	1
  (0, 3)	1
  (1, 0)	1
  (1, 3)	2
  (1, 5)	1
  (1, 12)	1
  (1, 13)	1
  (2, 1)	1
  (2, 3)	1
  (2, 7)	1
  (3, 4)	1
  (3, 8)	1
  (3, 9)	1
  (3, 14)	1
  (4, 8)	1
  (4, 9)	1
  (4, 10)	1
  (4, 11)	1
  (4, 14)	1
  (5, 2)	1
  (5, 6)	1
  (5, 9)	1
  (5, 14)	1


In [17]:
cv_matrix = cv_matrix.toarray()
cv_matrix

array([[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0],
       [0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1]])

Cada término único es una característica de la matriz generada:

In [18]:
# obtenemos palabras únicas en el corpus
vocab = cv.get_feature_names_out()
# mostramos vectores de características BoW del corpus
pd.DataFrame(cv_matrix, columns=vocab)

,azul,bonito,buenas,cielo,desayunado,encanta,están,hacía,huevos,jamón,juan,odia,pero,plomizo,tostadas
0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,2,0,1,0,0,0,0,0,0,1,1,0
2,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1
4,0,0,0,0,0,0,0,0,1,1,1,1,0,0,1
5,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1


In [19]:
#id de las palabras del vocabulario
cv.vocabulary_.get('cielo')

3

In [20]:
#si una palabra no está en el vocabulario...
cv.vocabulary_.get('lluvia')

### Aplicando el modelo a nuevos documentos
Cuando calculamos el vector BoW de un texto nuevo con el modelo no hay que volver a ajustar el vocabulario, por lo que los términos nuevos no se tendrán en cuenta:

In [21]:
nuevo_corpus = ['El Cielo amenaza lluvia', 'Pedro desayuna tostadas de jamón con tomate']
cv_matrix_nueva = cv.transform(map(normalizar_doc, nuevo_corpus))
cv_matrix_nueva

<2x15 sparse matrix of type '<class 'numpy.int64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [22]:
pd.DataFrame(cv_matrix_nueva.toarray(), columns=vocab)

,azul,bonito,buenas,cielo,desayunado,encanta,están,hacía,huevos,jamón,juan,odia,pero,plomizo,tostadas
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1


In [23]:
list(map(normalizar_doc, nuevo_corpus))

['cielo amenaza lluvia', 'pedro desayuna tostadas jamón tomate']

### Modelos N-grams
Considera como términos del vocabulario cada secuencia de N palabras consecutivas que aparece en el texto (*n-gramas*).  
Por ejemplo para los *bigrams* del corpus (N=2):

In [24]:
bv = CountVectorizer(ngram_range=(2,2))
bv_matrix = bv.fit_transform(norm_corpus)

In [25]:
bv_matrix

<6x17 sparse matrix of type '<class 'numpy.int64'>'
	with 19 stored elements in Compressed Sparse Row format>

In [26]:
bv.get_feature_names_out()

array(['azul bonito', 'azul pero', 'bonito cielo', 'cielo azul',
       'cielo hacía', 'cielo plomizo', 'desayunado huevos',
       'encanta cielo', 'están buenas', 'huevos jamón', 'jamón están',
       'jamón tostadas', 'juan odia', 'odia tostadas', 'pero cielo',
       'tostadas huevos', 'tostadas jamón'], dtype=object)

In [27]:
norm_corpus

['cielo azul bonito',
 'encanta cielo azul pero cielo plomizo',
 'bonito cielo hacía',
 'desayunado huevos jamón tostadas',
 'juan odia tostadas huevos jamón',
 'tostadas jamón están buenas']

In [28]:
len(bv.get_feature_names_out())

17

In [29]:
bv_matrix

<6x17 sparse matrix of type '<class 'numpy.int64'>'
	with 19 stored elements in Compressed Sparse Row format>

In [30]:
bv_matrix = bv_matrix.toarray()
vocab_bigram = bv.get_feature_names_out()
pd.DataFrame(bv_matrix, columns=vocab_bigram)

,azul bonito,azul pero,bonito cielo,cielo azul,cielo hacía,cielo plomizo,desayunado huevos,encanta cielo,están buenas,huevos jamón,jamón están,jamón tostadas,juan odia,odia tostadas,pero cielo,tostadas huevos,tostadas jamón
0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,1,0,1,0,1,0,0,0,0,0,0,1,0,0
2,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0
5,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1


Se puede establecer el rango de n-grams a `(1,2)` para obtener el conjunto de unigramas y bigramas del corpus.  
Para limitar el número de términos en el vocabulario del modelo BoW se puede limitar a los términos que aparecen en un mínimo de documentos con el parámetro `min_df`

In [31]:
bv = CountVectorizer(ngram_range=(1,2), min_df=2)
bv_matrix = bv.fit_transform(norm_corpus)

bv_matrix = bv_matrix.toarray()
vocab_bigram = bv.get_feature_names_out()
pd.DataFrame(bv_matrix, columns=vocab_bigram)

,azul,bonito,cielo,cielo azul,huevos,huevos jamón,jamón,tostadas
0,1,1,1,1,0,0,0,0
1,1,0,2,1,0,0,0,0
2,0,1,1,0,0,0,0,0
3,0,0,0,0,1,1,1,1
4,0,0,0,0,1,1,1,1
5,0,0,0,0,0,0,1,1


In [32]:
bv_matrix.shape

(6, 8)

### Ejercicio 1
Aplica el último modelo de BoW (unigramas y bigramas con min_df=2) al nuevo corpus de texto

In [33]:
#completar
bv_matrix = bv.transform(map(normalizar_doc, nuevo_corpus))
pd.DataFrame(bv_matrix.toarray(), columns=vocab_bigram)

,azul,bonito,cielo,cielo azul,huevos,huevos jamón,jamón,tostadas
0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,1,1


# Librería `Gensim`
Para trabajar con la librería `Gensim` es necesario transformar los documentos en una lista de tokens.

Convertimos nuestros texto de ejemplo en una lista de tokens y visualizamos el primer documento como ejemplo:

In [34]:
def normalizar_doc_tokenize(doc):
    '''Función que normaliza un texto cogiendo sólo
    las palabras en minúsculas mayores de 3 caracteres'''
    # separamos en tokens
    tokens = nlp(doc)
    # filtramos stopwords
    filtered_tokens = [t.lower_ for t in tokens if
                       len(t.text)>3 and
                       not t.is_space and
                       not t.is_punct]

    return filtered_tokens

In [35]:
normalizar_doc_tokenize(corpus[0])

['cielo', 'azul', 'bonito']

In [36]:
normalizar_doc(corpus[0])

'cielo azul bonito'

In [37]:
tokenized_corpus = [normalizar_doc_tokenize(doc) for doc in corpus]
tokenized_corpus

[['cielo', 'azul', 'bonito'],
 ['encanta', 'cielo', 'azul', 'pero', 'cielo', 'plomizo'],
 ['bonito', 'cielo', 'hacía'],
 ['desayunado', 'huevos', 'jamón', 'tostadas'],
 ['juan', 'odia', 'tostadas', 'huevos', 'jamón'],
 ['tostadas', 'jamón', 'están', 'buenas']]

In [38]:
#con la función map:
list(map(normalizar_doc_tokenize, corpus))

[['cielo', 'azul', 'bonito'],
 ['encanta', 'cielo', 'azul', 'pero', 'cielo', 'plomizo'],
 ['bonito', 'cielo', 'hacía'],
 ['desayunado', 'huevos', 'jamón', 'tostadas'],
 ['juan', 'odia', 'tostadas', 'huevos', 'jamón'],
 ['tostadas', 'jamón', 'están', 'buenas']]

## Modelo Bag of Words
Se pasará al modelo de Gensim como:

In [39]:
from gensim.corpora import Dictionary

diccionario = Dictionary(tokenized_corpus) #también funcionaría Dictionary(map(normalizar_doc_tokenize, corpus))

In [40]:
diccionario

In [41]:
dir(diccionario)

['__abstractmethods__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_adapt_by_suffix',
 '_load_specials',
 '_save_specials',
 '_smart_save',
 'add_documents',
 'add_lifecycle_event',
 'cfs',
 'compactify',
 'dfs',
 'doc2bow',
 'doc2idx',
 'filter_extremes',
 'filter_n_most_frequent',
 'filter_tokens',
 'from_corpus',
 'from_documents',
 'get',
 'id2token',
 'items',
 'iteritems',
 'iterkeys',
 'itervalues',
 'keys',
 'lifecycle_events',
 'load',
 'load_from_text',
 'merge_with',
 'most_common',
 'num_docs',
 'num_nnz',
 'num_pos',
 'patch_wi

El ID de cada palabra del diccionario se obtiene con:

In [42]:
diccionario.token2id

{'azul': 0,
 'bonito': 1,
 'cielo': 2,
 'encanta': 3,
 'pero': 4,
 'plomizo': 5,
 'hacía': 6,
 'desayunado': 7,
 'huevos': 8,
 'jamón': 9,
 'tostadas': 10,
 'juan': 11,
 'odia': 12,
 'buenas': 13,
 'están': 14}

In [43]:
len(diccionario.token2id)

15

La librería `gensim` crea la matriz BoW con otro formato. A cada palabra distinta del corpus se le asigna un ID único. Por cada documento se genera una lista de tuplas (ID, frecuencia) con la frecuencia de aparición de cada palabra:

In [44]:
diccionario.doc2bow(tokenized_corpus[0])

[(0, 1), (1, 1), (2, 1)]

In [45]:
diccionario.token2id['plomizo'] #ID de cada término

5

In [46]:
diccionario[5] #término correspondiente a una ID

'plomizo'

In [47]:
diccionario.id2token #diccionario de palabras para cada ID

{0: 'azul',
 1: 'bonito',
 2: 'cielo',
 3: 'encanta',
 4: 'pero',
 5: 'plomizo',
 6: 'hacía',
 7: 'desayunado',
 8: 'huevos',
 9: 'jamón',
 10: 'tostadas',
 11: 'juan',
 12: 'odia',
 13: 'buenas',
 14: 'están'}

In [48]:
for (i, tf) in diccionario.doc2bow(tokenized_corpus[0]):
    print(f"{diccionario[i]}: {tf}")

azul: 1
bonito: 1
cielo: 1


In [49]:
mapped_corpus = [diccionario.doc2bow(text)
                 for text in tokenized_corpus]

In [50]:
mapped_corpus

[[(0, 1), (1, 1), (2, 1)],
 [(0, 1), (2, 2), (3, 1), (4, 1), (5, 1)],
 [(1, 1), (2, 1), (6, 1)],
 [(7, 1), (8, 1), (9, 1), (10, 1)],
 [(8, 1), (9, 1), (10, 1), (11, 1), (12, 1)],
 [(9, 1), (10, 1), (13, 1), (14, 1)]]

In [51]:
len(mapped_corpus)

6

### Ejercicio 2
Calcula la matriz BoW de `mapped_corpus` directamente del Corpus original en `corpus` usando la función `map`

In [52]:
mapped_corpus = [diccionario.doc2bow(text)
                 for  text in map (normalizar_doc_tokenize, corpus)]

mapped_corpus

[[(0, 1), (1, 1), (2, 1)],
 [(0, 1), (2, 2), (3, 1), (4, 1), (5, 1)],
 [(1, 1), (2, 1), (6, 1)],
 [(7, 1), (8, 1), (9, 1), (10, 1)],
 [(8, 1), (9, 1), (10, 1), (11, 1), (12, 1)],
 [(9, 1), (10, 1), (13, 1), (14, 1)]]

In [53]:
#frec. de documentos de cada token
diccionario.dfs

{2: 3,
 0: 2,
 1: 2,
 3: 1,
 4: 1,
 5: 1,
 6: 1,
 7: 1,
 8: 2,
 9: 3,
 10: 3,
 11: 1,
 12: 1,
 14: 1,
 13: 1}

Convirtiendo los ID en su término correspondiente:

In [54]:
for i in diccionario.dfs:
    print(f"{diccionario[i]}: {diccionario.dfs[i]}")

cielo: 3
azul: 2
bonito: 2
encanta: 1
pero: 1
plomizo: 1
hacía: 1
desayunado: 1
huevos: 2
jamón: 3
tostadas: 3
juan: 1
odia: 1
están: 1
buenas: 1


In [55]:
#frec. aparición total de cada token
diccionario.cfs

{2: 4,
 0: 2,
 1: 2,
 3: 1,
 4: 1,
 5: 1,
 6: 1,
 7: 1,
 8: 2,
 9: 3,
 10: 3,
 11: 1,
 12: 1,
 14: 1,
 13: 1}

### Ejercicio 3
Recorre el diccionario `cfs` mostrando el término correspondiente para cada ID y su frecuencia

In [56]:
for i in diccionario.dfs:
    print(f'{diccionario[i]}: {diccionario.dfs[i]}')

cielo: 3
azul: 2
bonito: 2
encanta: 1
pero: 1
plomizo: 1
hacía: 1
desayunado: 1
huevos: 2
jamón: 3
tostadas: 3
juan: 1
odia: 1
están: 1
buenas: 1


## Aplicación de los modelos a nuevos textos
Para aplicar un modelo BoW o TF-IDF a un nuevo documento hay que utilizar los modelos ya entrenados en `gensim` sobre el corpus original
### Modelo BoW

In [57]:
tokenized_nuevo_corpus = [normalizar_doc_tokenize(doc) for doc in nuevo_corpus]

mapped_nuevo_corpus = [diccionario.doc2bow(text)
                 for text in tokenized_nuevo_corpus]

mapped_nuevo_corpus

[[(2, 1)], [(9, 1), (10, 1)]]

In [58]:
len(mapped_nuevo_corpus)

2

In [59]:
for (i, tf) in mapped_nuevo_corpus[0]:
    print(f"{diccionario[i]}: {tf}")

cielo: 1


In [60]:
for (i, tf) in mapped_nuevo_corpus[1]:
    print(f"{diccionario[i]}: {tf}")

jamón: 1
tostadas: 1


In [61]:
#Más pythonico con 'map'
list(map(lambda x: diccionario.doc2bow(normalizar_doc_tokenize(x)), nuevo_corpus))

[[(2, 1)], [(9, 1), (10, 1)]]

### Ejercicio 4
Define una función que devuelva el BoW de una lista de nuevos textos (pasada como lista de *strings*) usando el diccionario y la función de normalización creadas anteriormente.

In [62]:
def bow(corpus, diccionario=diccionario, normalizacion=normalizar_doc_tokenize):
    """Genera la matriz BoW de la lista de texto en 'corpus'
    usando el diccionario y la función de normalización
    pasados como argumentos
    Devuelve una lista de vectores BoW"""
    
    #COMPLETAR
    return list(map(lambda x: diccionario.doc2bow(normalizacion(x)),corpus))

In [63]:
bow(nuevo_corpus)

[[(2, 1)], [(9, 1), (10, 1)]]

## Uso de generadores
Para no tener que crear una lista intermedia con todo el corpus normalizado (y tokenizado en Gensim) podemos usar una función de tipo *generador* para alimentar el vectorizador BoW

In [64]:
def build_texts(fname):
    """
    Generador que devuelve el texto normalizado a partir de un archivo
    línea a línea
    """
    with open(fname) as f:
        for line in f:
            yield normalizar_doc(line)

In [65]:
build_texts('lee_background.cor')

<generator object build_texts at 0x7f56aec60190>

In [66]:
corpus = build_texts('lee_background.cor')

In [67]:
next(corpus)

"hundreds people have been forced vacate their homes southern highlands south wales strong winds today pushed huge bushfire towards town hill blaze near goulburn south-west sydney forced closure hume highway about 4:00pm aedt marked deterioration weather storm cell moved east across blue mountains forced authorities make decision evacuate people from homes outlying streets hill south wales southern highlands estimated residents have left their homes nearby mittagong south wales rural fire service says weather conditions which caused fire burn finger formation have eased about fire units around hill optimistic defending properties more than blazes burn year's south wales fire crews have been called fire gunning south goulburn while details available this stage fire authorities says closed hume highway both directions meanwhile fire sydney's west longer threatening properties cranebrook area rain fallen some parts illawarra sydney hunter valley north coast bureau meteorology's claire ric

In [68]:
cv = CountVectorizer()
bow_gen = cv.fit_transform(corpus)

In [69]:
bow_gen

<299x6847 sparse matrix of type '<class 'numpy.int64'>'
	with 28475 stored elements in Compressed Sparse Row format>

Cuidado porque el primer documento no lo procesa (ya lo hemos usado)

In [70]:
bow_gen = cv.fit_transform(build_texts('lee_background.cor'))

### Ejercicio 5
Crea una función de tipo *generador* para leer y tokenizar un corpus en archivo (un documento por línea) y luego crea su matriz BoW con la librería Gensim

In [71]:
def build_texts_token(fname):
    """generador que procesa linea a linea un archivo
    y devuelve cada texto normalizado y tokenizado"""

    with open(fname) as f:
        for line in f:
            yield normalizar_doc_tokenize(line)   

In [72]:
next(build_texts_token('lee_background.cor'))

['hundreds',
 'people',
 'have',
 'been',
 'forced',
 'vacate',
 'their',
 'homes',
 'southern',
 'highlands',
 'south',
 'wales',
 'strong',
 'winds',
 'today',
 'pushed',
 'huge',
 'bushfire',
 'towards',
 'town',
 'hill',
 'blaze',
 'near',
 'goulburn',
 'south-west',
 'sydney',
 'forced',
 'closure',
 'hume',
 'highway',
 'about',
 '4:00pm',
 'aedt',
 'marked',
 'deterioration',
 'weather',
 'storm',
 'cell',
 'moved',
 'east',
 'across',
 'blue',
 'mountains',
 'forced',
 'authorities',
 'make',
 'decision',
 'evacuate',
 'people',
 'from',
 'homes',
 'outlying',
 'streets',
 'hill',
 'south',
 'wales',
 'southern',
 'highlands',
 'estimated',
 'residents',
 'have',
 'left',
 'their',
 'homes',
 'nearby',
 'mittagong',
 'south',
 'wales',
 'rural',
 'fire',
 'service',
 'says',
 'weather',
 'conditions',
 'which',
 'caused',
 'fire',
 'burn',
 'finger',
 'formation',
 'have',
 'eased',
 'about',
 'fire',
 'units',
 'around',
 'hill',
 'optimistic',
 'defending',
 'properties',
 'm

In [73]:
#creamos el diccionario
dict = Dictionary(build_texts_token('lee_background.cor'))

In [74]:
len(dict.token2id)

7211

In [76]:
#construimos la matriz BoW
bow = [dict.doc2bow(t) for t in build_texts_token('lee_background.cor')]

In [77]:
len(bow)

300